# MAKE_GRID Function Explained

**Description:**

make_grid is a function used in a number of public HuBMAP notebooks for inference and creating a submission.
Seemed to me this function was essential to making a submission, and so I thought I would comb through the function line by line to get a better understanding of how it works.

**References:**

https://www.kaggle.com/leighplt/pytorch-fcn-resnet50
https://www.kaggle.com/joshi98kishan/hubmap-keras-pipeline-training-inference
https://www.kaggle.com/c/hubmap-kidney-segmentation/overview/supervised-ml-evaluation

**Function Expectation:**

 - "Return Array of size (N,4), where N - number of tiles, 2nd axis represente slices: x1,x2,y1,y2"
 - The function breaks the image into blocks.  In our case 1024x1024 blocks (which are later resized to 256x256).
 - Technically, make_grid just specifies the blocks via a series of pixel coordinates ((x1,y1),(x2,y2)).
 - Later, we use the output to loop through each set of coordinates, extract the image, resize, predict, and store results.

# Imports

In [ ]:
# For reading tiff images in parts 
import rasterio
from rasterio.windows import Window
import pathlib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# Full 'Make_Grid' Function

In [ ]:
# https://www.kaggle.com/leighplt/pytorch-fcn-resnet50
def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N = number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4)

# Settings & Global Variables

In [ ]:
DATA_ORIG_PATH = '../input/hubmap-kidney-segmentation'
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
# WINDOW is the size of the tile to be read by rasterio
WINDOW = 1024
# Tiles will have some overlap
MIN_OVERLAP = 32
# Tiles will be resized to NEW_SIZE, which is the size of the image
# on which, we have trained our model.
NEW_SIZE = 256

https://docs.python.org/3/library/pathlib.html

"This module offers classes representing filesystem paths with semantics appropriate for different operating systems. Path classes are divided between pure paths, which provide purely computational operations without I/O, and concrete paths, which inherit from pure paths but also provide I/O operations."

In [ ]:
p = pathlib.Path(DATA_ORIG_PATH)
p

empty dictionary for submission

In [ ]:
subm = {}

we're going to iterate through each test image

In [ ]:
p.glob('test/*.tiff')

let's take a look at the beginning of the inference loop.

* We start by reading in an image
* Next we run 'make_grid' on an image, and we get a numpy array returned of int values with shape 1710x4

In [ ]:
for i, filename in enumerate(p.glob('test/*.tiff')):
    print("i: ", i)
    print("filename: ", filename)
    print(f'{i+1} Predicting {filename.stem}')
    dataset = rasterio.open(filename.as_posix(), transform = identity)
    print("dataset: ", dataset)
    print("dataset shape: ", dataset.shape)
    slices = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)
    print("slices shape: ", slices.shape)
    print("slices type: ", type(slices))
    print("first 5 values of slices: ", slices[:5])
    break

So let's look at make_grid line by line.  As input the function takes...

* the image size in form tuple
* a window size which we've pre-selected
* and a min_overlap value which we've pre-selected

In [ ]:
shape = (36800, 43780)
window=WINDOW
min_overlap=MIN_OVERLAP

Next we unwrap shape into values x and y

In [ ]:
x, y = shape
print(x)
print(y)

In [ ]:
print("x: ", x)
print("window: ", window)
print("min_overlap: ", min_overlap)
print("window - min_overlap: ", window - min_overlap)
print("{} / {} = {}".format(x,window - min_overlap,x / (window - min_overlap)))
print("{} // {} i.e. int division = {}".format(x,window - min_overlap,x // (window - min_overlap)))
nx = x // (window - min_overlap) + 1
print("nx i.e. int division result + 1 = ", nx)

numpy.linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None, axis=0)[source]¶
Return evenly spaced numbers over a specified interval.

### **Returns num evenly spaced samples, calculated over the interval [start, stop].**

The endpoint of the interval can optionally be excluded.

Changed in version 1.16.0: Non-scalar start and stop are now supported.

Parameters

start

array_like
The starting value of the sequence.

stop

array_like
The end value of the sequence, unless endpoint is set to False. In that case, the sequence consists of all but the last of num + 1 evenly spaced samples, so that stop is excluded. Note that the step size changes when endpoint is False.

num

int, optional
Number of samples to generate. Default is 50. Must be non-negative.

endpoint

bool, optional
If True, stop is the last sample. Otherwise, it is not included. Default is True.

retstep

bool, optional
If True, return (samples, step), where step is the spacing between samples.

dtyped

type, optional
The type of the output array. If dtype is not given, infer the data type from the other input arguments.

New in version 1.9.0.

axisint, optional
The axis in the result to store the samples. Relevant only if start or stop are array-like. By default (0), the samples will be along a new axis inserted at the beginning. Use -1 to get an axis at the end.

New in version 1.16.0.

Returns
samples

ndarray
There are num equally spaced samples in the closed interval [start, stop] or the half-open interval [start, stop) (depending on whether endpoint is True or False).

stepfloat, optional
Only returned if retstep is True

Size of spacing between samples.

In [ ]:
x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
print("len(x1): ", len(x1))
print("x1: ", x1)

now we take the last values in x1 and set it to x-window

In [ ]:
print("original value of x1[-1]: ", x1[-1])
print("x: ", x)
print("window: ", window)
x1[-1] = x - window
print("new value of x1[-1]", x1[-1])

next we set x2 to x1 + window

In [ ]:
print("x1: ", x1)
print("window:", window)
print("x1 + window: ", x1 + window)
x2 = (x1 + window).clip(0, x)
print("x2: ", x2)

then we complete the same process for 'y'

In [ ]:
ny = y // (window - min_overlap) + 1
print("ny: ", ny)
y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
y1[-1] = y - window
print("y1: ", y1)
y2 = (y1 + window).clip(0, y)
print("y2: ", y2)

initialize a numpy matrix 'slices' using nx & ny

In [ ]:
slices = np.zeros((nx,ny, 4), dtype=np.int64)
print(slices)

fill slices with our values from x1, x2, y1, y2

In [ ]:
for i in range(nx):
    for j in range(ny):
        slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
print(slices)

reshape

In [ ]:
slices.reshape(nx*ny,4)
print(slices)

the full loop!

In [ ]:
# for i, filename in tqdm(enumerate(p.glob('test/*.tiff')), 
#                         total = len(list(p.glob('test/*.tiff')))):
    
#     print(f'{i+1} Predicting {filename.stem}')
    
#     dataset = rasterio.open(filename.as_posix(), transform = identity)
#     slices = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)
#     preds = np.zeros(dataset.shape, dtype=np.uint8)
    
#     for (x1,x2,y1,y2) in slices:
#         image = dataset.read([1,2,3],
#                     window=Window.from_slices((x1,x2),(y1,y2)))
#         image = np.moveaxis(image, 0, -1)
        
#         image = tf.image.convert_image_dtype(image, 
#                                  tf.float32)
#         image = cv2.resize(image.numpy(), (NEW_SIZE, NEW_SIZE))
#         image = np.expand_dims(image, 0)
        
#         pred = None
        
#         for fold_model in trained_fold_models:
#             if pred is None:
#                 pred = np.squeeze(fold_model.predict(image))
#             else:
#                 pred += np.squeeze(fold_model.predict(image))
        
#         pred = pred/len(trained_fold_models)
        
#         pred = cv2.resize(pred, (WINDOW, WINDOW))
#         preds[x1:x2,y1:y2] = (pred > 0.5).astype(np.uint8)
            
#     subm[i] = {'id':filename.stem, 'predicted': rle_numba_encode(preds)}
#     del preds
#     gc.collect();

# The End

Hope this helps some people as it helped me.  :-)